In [1]:
import random, shutil, os
import numpy as np
import pandas as pd

from mido import Message, MidiFile, MidiTrack
from glob import glob

In [2]:
def create_only_folders(dir, files):
    return [f for f in files if os.path.isfile(os.path.join(dir,f))]


shutil.copytree(r"C:\Users\KIST\Desktop\MIDI", './bar_dataset_2', ignore=create_only_folders)

'./bar_dataset_2'

In [3]:
midi_list = glob(r"C:\Users\KIST\Desktop\MIDI\**\**\**\*.mid")

In [4]:
def classify(note, midi):
    
    cat = ''

    if 'gmd' in midi.lower():

        if note in [36]: # kick
            cat = 0
        elif note in [38, 40, 37]: # snare 
            cat = 1
        elif note in [48, 50]: # High Tom
            cat = 2
            # cat = 2 # Replace HT -> MT
        elif note in [45, 47]: # Mid Tom
            cat = 3
            # cat = 2
        elif note in [43, 58]: # Low Tom
            cat = 4
            # cat = 3
        elif note in [46, 26]: # Open Hat
            cat = 5
            # cat = 4
        elif note in [42, 22, 44]: # Closed Hat
            cat = 6
            # cat = 5
        elif note in [49, 55, 57, 52]: # Crash
            cat = 7 
            # cat = 6 # Replace Cr -> Ri
        elif note in [51, 59, 53]: # Ride
            # cat = 6
            cat = 8
        else:
            cat = 9
    
    else:
        if note in [35, 36]: # kick
            cat = 0
        elif note in [37, 38, 39, 40]: # snare 
            cat = 1
        elif note in [48, 50]: # High Tom
            # cat = 2
            cat = 2 # Replace HT -> MT
        elif note in [45, 47]: # Mid Tom
            cat = 3
            # cat = 2
        elif note in [41, 43]: # Low Tom
            cat = 4
            # cat = 3
        elif note in [46]: # Open Hat
            cat = 5
            # cat = 4
        elif note in [42, 44]: # Closed Hat
            cat = 6
            # cat = 5
        elif note in [49, 55, 57, 52]: # Crash
            cat = 7 
            # cat = 6 # Replace Cr -> Ri
        elif note in [51, 59, 53]: # Ride
            # cat = 6
            cat = 8
        else:
            cat = 9

    return cat

def gm_kit_classify(note):
    cat = ''
    if note in [35, 36]: # kick
        cat = 0
    elif note in [37, 38, 39, 40]: # snare 
        cat = 1
    elif note in [48, 50]: # High Tom
        cat = 2
        # cat = 2 # Replace HT -> MT
    elif note in [45, 47]: # Mid Tom
        cat = 3
        # cat = 2
    elif note in [41, 43]: # Low Tom
        cat = 4
        # cat = 3
    elif note in [46]: # Open Hat
        cat = 5
        # cat = 4
    elif note in [42, 44]: # Closed Hat
        cat = 6
        # cat = 5
    elif note in [49, 55, 57, 52]: # Crash
        cat = 7 
        # cat = 6 # Replace Cr -> Ri
    elif note in [51, 59, 53]: # Ride
        # cat = 6
        cat = 8
    else:
        cat = 9

    return cat

def in_gmd(note):
    if note in [36, 38, 40, 37, 48, 50, 45, 47, 43, 58, 46, 26, 42, 22, 44, 49, 55, 57, 52, 51, 59, 53]:
        return True
    else:
        return False

def in_gm(note):
    if note in [35, 36, 37, 38, 39, 40, 48, 50, 45, 47, 41, 43, 46, 42, 44, 49, 55, 57, 52, 51, 59, 53]:
        return True
    else:
        return False
        

In [6]:
# mid2 = MidiFile('/Users/cooky/Desktop/groove/drummer7/session1/3_rock_86_fill_4-4.mid', clip=True)
cnt = 0

for midi in midi_list:
    
    mid = MidiFile(midi, clip=True)
    on_sec = 0
    event = []
    quantize = 120
    if 'monkee' in midi.lower():
        quantize = 60
    elif 'longloops2' in midi.lower():
        quantize = 24

    if 'fill' not in midi.lower():
        # print(midi)
    
        for msg in mid.tracks[mid.tracks.__len__() - 1]:

            note = str(msg).split()

            if len(note) == 5:
                on_sec += msg.dict()['time']

                if note[0] == 'note_on':
                    if [round(on_sec/120), msg.dict()['note']] not in event:
                        event.append([round(on_sec/120), msg.dict()['note']])
        # print(midi)
        # print(event)
        beat = pd.DataFrame(0, columns = np.arange(10), index = np.arange(event[-1][0] + 1))

        for e in event:
            beat.loc[e[0], classify(e[1], midi=midi)] = 1
        

        beat.to_csv('./midi2df/beat/' + str(cnt) + '.csv.gzip', index=False, compression='gzip')
        cnt += 1
            


In [7]:
monkee_midi_list = glob(r"C:\Users\KIST\Desktop\MIDI\Groove*\**\**\*.mid")

In [8]:
asd = []
for i in monkee_midi_list:
    if i[-10:-6] == 'Fill':
        asd.append(i[:-11])

In [9]:
asd = list(set(asd))

In [10]:
cnt = 1

for i in asd:
    fill = []
    beat = []
    for j in monkee_midi_list:
        if i in j:
            if 'fill' in j.lower():
                fill.append(j)
            else:
                beat.append(j)
    
    for b in beat:
        for f in fill:
            b_midi = MidiFile(b, clip=True)
            f_midi = MidiFile(f, clip=True)
            b_on_sec = 0
            f_on_sec = 0
            b_event = []
            f_event = []

            for msg in b_midi.tracks[b_midi.tracks.__len__() - 1]:
                note = str(msg).split()
                if len(note) == 5:
                    b_on_sec += msg.dict()['time']

                    if note[0] == 'note_on':
                        if [round(b_on_sec/60), msg.dict()['note']] not in b_event:
                            b_event.append([round(b_on_sec/60), msg.dict()['note']])
            
            for msg in f_midi.tracks[f_midi.tracks.__len__() - 1]:
                note = str(msg).split()
                if len(note) == 5:
                    f_on_sec += msg.dict()['time']

                    if note[0] == 'note_on':
                        if [round(f_on_sec/60), msg.dict()['note']] not in f_event:
                            f_event.append([round(f_on_sec/60), msg.dict()['note']])
                
            if b_event[-1][0]%16 != 0:
                add = 1
            else:
                add = 0
            b_beat = pd.DataFrame(0, columns = np.arange(10), index = np.arange((b_event[-1][0]//16 + add) * 16))
            f_beat = pd.DataFrame(0, columns = np.arange(10), index = np.arange((f_event[-1][0] + 1)))

            for e in b_event:
                b_beat.loc[e[0], classify(e[1], midi=b)] = 1
            if (b_beat[-15:] == np.zeros((15,10))).all().all():
                b_beat = b_beat[:-16]

            for e in f_event:
                f_beat.loc[e[0], classify(e[1], midi=f)] = 1

            bf = pd.concat([b_beat, f_beat])

            print(b)
            print(f)
            bf.to_csv('./midi2df/fill/' + str(cnt) + '.csv.gzip', index = False, compression = 'gzip')
            cnt += 1



            




C:\Users\KIST\Desktop\MIDI\Groove Monkee Variety Pack\Variety Pack GM\Rock\140 Half-Time Crash.mid
C:\Users\KIST\Desktop\MIDI\Groove Monkee Variety Pack\Variety Pack GM\Rock\140 Half-Time Fill 1.mid
C:\Users\KIST\Desktop\MIDI\Groove Monkee Variety Pack\Variety Pack GM\Rock\140 Half-Time Crash.mid
C:\Users\KIST\Desktop\MIDI\Groove Monkee Variety Pack\Variety Pack GM\Rock\140 Half-Time Fill 2.mid
C:\Users\KIST\Desktop\MIDI\Groove Monkee Variety Pack\Variety Pack GM\Rock\140 Half-Time Crash.mid
C:\Users\KIST\Desktop\MIDI\Groove Monkee Variety Pack\Variety Pack GM\Rock\140 Half-Time Fill 3.mid
C:\Users\KIST\Desktop\MIDI\Groove Monkee Variety Pack\Variety Pack GM\Rock\140 Half-Time Crash.mid
C:\Users\KIST\Desktop\MIDI\Groove Monkee Variety Pack\Variety Pack GM\Rock\140 Half-Time Fill 4.mid
C:\Users\KIST\Desktop\MIDI\Groove Monkee Variety Pack\Variety Pack GM\Rock\140 Half-Time Crash.mid
C:\Users\KIST\Desktop\MIDI\Groove Monkee Variety Pack\Variety Pack GM\Rock\140 Half-Time Fill 5.mid
C:\Us

In [11]:
(asd[-15:] == np.zeros((15,8))).all().all()

C:\Users\KIST\AppData\Local\Temp\ipykernel_14232\3777783374.py:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  (asd[-15:] == np.zeros((15,8))).all().all()


AttributeError: 'bool' object has no attribute 'all'

In [12]:
np.zeros((15,8))

array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])

In [38]:
a = pd.read_csv('./bar_dataset2/merged_beat/44.csv.gzip', compression='gzip')

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(a)

    0  1  2  3  4  5  6  7
0   1  0  0  0  0  0  1  0
1   0  0  0  0  0  0  0  0
2   0  0  0  0  0  0  1  0
3   0  0  0  0  0  0  0  0
4   0  1  0  0  0  0  1  0
5   0  0  0  0  0  0  0  0
6   0  1  0  0  0  0  1  0
7   0  1  0  0  0  0  0  0
8   1  0  0  0  0  0  1  0
9   0  1  0  0  0  0  0  0
10  1  0  0  0  0  0  1  0
11  0  0  0  0  0  0  0  0
12  0  1  0  0  0  0  1  0
13  0  0  0  0  0  0  0  0
14  0  0  0  0  0  0  1  0
15  0  0  0  0  0  0  0  0
16  1  0  0  0  0  0  1  0
17  0  1  0  0  0  0  0  0
18  1  0  0  0  0  0  1  0
19  0  0  0  0  0  0  0  0
20  0  1  0  0  0  0  1  0
21  0  0  0  0  0  0  0  0
22  0  1  0  0  0  0  1  0
23  0  1  0  0  0  0  0  0
24  1  0  0  0  0  0  1  0
25  0  1  0  0  0  0  0  0
26  1  0  0  0  0  0  1  0
27  0  0  0  0  0  0  0  0
28  0  1  0  0  0  0  1  0
29  0  0  0  0  0  0  0  0
30  1  0  0  0  0  0  1  0
31  0  0  0  0  0  0  0  0
32  1  0  0  0  0  0  1  0
33  0  0  0  0  0  0  0  0
34  0  0  0  0  0  0  1  0
35  0  0  0  0  0  0  0  0
3

In [14]:
beat_gzip = glob('./midi2df/beat/*.csv.gzip')

In [15]:
seven_kind_arr = [
   # bd sn mt lt oh ch ri 
    [1, 0, 0, 0, 0, 0, 0], #0
    [0, 1, 0, 0, 0, 0, 0], #1
    [0, 0, 1, 0, 0, 0, 0], #2
    [0, 0, 0, 1, 0, 0, 0], #3
    [0, 0, 0, 0, 1, 0, 0], #4
    [0, 0, 0, 0, 0, 1, 0], #5
    [0, 0, 0, 0, 0, 0, 1], #6
    [1, 0, 0, 0, 0, 1, 0], #7
    [1, 0, 0, 0, 1, 0, 0], #8
    [1, 0, 0, 0, 0, 0, 1], #9
    [1, 0, 0, 1, 0, 0, 0], #10
    [1, 1, 0, 0, 0, 0, 0], #11
    [0, 1, 0, 0, 0, 1, 0], #12
    [0, 1, 0, 0, 1, 0, 0], #13
    [0, 1, 0, 0, 0, 0, 1], #14
    [0, 1, 0, 1, 0, 0, 0], #15
    [0, 1, 0, 0, 0, 0, 1], #16
    [0, 0, 1, 1, 0, 0, 0], #17
    [1, 1, 0, 0, 0, 1, 0], #18
    [1, 1, 0, 0, 1, 0, 0], #19
    [1, 1, 0, 0, 0, 0, 1], #20
    [0, 0, 0, 0, 0, 0, 0]
]


# bd sn ht mt lt oh ch cr ri
checking_arr = [
    [0, 0, 0, 0, 0, 0, 0, 0, 0], # 0 null
    [1, 0, 0, 0, 0, 0, 0, 0, 0], # 1  B
    [1, 0, 0, 0, 0, 0, 0, 1, 0], # 2  B+C
    [1, 0, 0, 0, 0, 0, 1, 0, 0], # 3  B+CH
    [1, 0, 1, 0, 0, 0, 0, 0, 0], # 4  B+HT
    [1, 0, 0, 0, 1, 0, 0, 0, 0], # 5  B+LT
    [1, 0, 0, 1, 0, 0, 0, 0, 0], # 6  B+MT
    [1, 0, 0, 0, 0, 1, 0, 0, 0], # 7  B+OH
    [1, 0, 0, 0, 0, 0, 0, 0, 1], # 8  B+R
    [0, 0, 0, 0, 0, 0, 1, 0, 0], # 9  CH
    [0, 0, 1, 0, 0, 0, 1, 0, 0], # 10 CH+HT
    [0, 0, 0, 0, 1, 0, 1, 0, 0], # 11 CH+LT
    [0, 0, 0, 0, 0, 0, 1, 0, 1], # 12 CH+R
    [0, 0, 1, 0, 0, 0, 0, 0, 0], # 13 HT
    [0, 0, 1, 0, 1, 0, 0, 0, 0], # 14 HT+LT
    [0, 0, 0, 0, 1, 0, 0, 0, 0], # 15 LT
    [0, 0, 0, 1, 0, 0, 0, 0, 0], # 16 MT
    [0, 0, 0, 0, 0, 1, 0, 0, 0], # 17 OH
    [0, 0, 0, 0, 0, 0, 0, 0, 1], # 18 R
    [0, 1, 0, 0, 0, 0, 0, 0, 0], # 19 S
    [1, 1, 0, 0, 0, 0, 0, 0, 0], # 20 S+B
    [1, 1, 0, 0, 0, 0, 1, 0, 0], # 21 S+B+CH
    [1, 1, 0, 0, 1, 0, 0, 0, 0], # 22 S+B+LT
    [1, 1, 0, 0, 0, 1, 0, 0, 0], # 23 S+B+OH
    [1, 1, 0, 0, 0, 0, 0, 0, 1], # 24 S+B+R
    [0, 1, 0, 0, 0, 0, 0, 1, 0], # 25 S+C
    [0, 1, 0, 0, 0, 0, 1, 0, 0], # 26 S+CH
    [0, 1, 0, 0, 1, 0, 0, 0, 0], # 27 S+LT
    [0, 1, 0, 0, 0, 1, 0, 0, 0], # 28 S+OH
    [0, 1, 0, 0, 0, 0, 0, 0, 1], # 29 S+R
    # [0, 0, 0, 0, 0, 0, 0, 1, 0], # 30 C
]


In [20]:
cnt = 1
for i in range(1,1263):
    asd = pd.read_csv('./midi2df/fill/' + str(i) + '.csv.gzip', compression='gzip')
    
    if asd.shape[0] >= 32:
        for _ in range(asd.shape[0]//32 + 1):
            useful = True
            rng = random.randint(0, asd.shape[0]-32)
            imsi = asd[rng:rng+32]
            asdd = imsi.drop('7', axis=1)
            for i in np.array(asdd):
                if list(i) not in checking_arr:
                    useful = False
            if useful:
                asdd.to_csv('./new_2bar/fill_' + str(cnt) + '.csv.gzip', compression='gzip', index=False)
                cnt += 1
            
            # asdd.to_csv('./new_2bar_non_filter/' + str(cnt) + '.csv.gzip', compression='gzip', index=False)
            # cnt += 1

In [32]:
for midi in os.listdir('./new_2bar/'):
    csv = pd.read_csv('./new_2bar/' + midi, compression='gzip').to_numpy()
    seq = []
    for note in csv:
        seq.append(checking_arr.index(note.tolist()))
        
        a = pd.DataFrame(np.zeros((32,29)))
        
        tmp = 0
        for i in seq:
            a.iloc[tmp,int(i)] = 1

            if tmp == 30:
                a.to_csv('./bar2singlelabel_masked/' + midi, index=False, compression='gzip')
            tmp += 1
    a.to_csv('./bar2singlelabel/'+midi, index=False, compression='gzip')
    

In [26]:
ass

,0,1,2,3,4,5,6,8,9
0,1,0,0,0,0,0,1,0,0
1,1,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,1,0,0
4,0,0,0,0,0,0,1,0,0
5,0,0,0,0,0,0,1,0,0
6,0,0,0,0,0,0,0,0,0
7,1,0,0,0,0,0,1,0,0
8,0,0,0,0,0,0,1,0,0
9,1,0,0,0,0,0,1,0,0


In [246]:
for i in np.array(asdd):
    if list(i) not in seven_kind_arr:
        print(i)

[1 0 0 0 0 1 1]
[0 0 0 0 0 1 1]
[0 0 1 0 0 0 1]
[0 0 0 0 0 1 1]
[0 1 0 0 0 1 1]
[1 0 1 0 0 1 1]
[0 0 0 1 0 0 1]
[0 0 0 0 0 1 1]
[0 0 1 0 0 1 0]
[0 1 0 0 0 1 1]
[1 0 1 0 0 1 1]
[0 0 0 0 0 1 1]
[0 0 0 0 0 1 1]
[0 0 1 0 0 1 1]
[1 0 0 0 0 1 1]
[0 0 1 0 0 1 0]
[0 1 0 0 0 1 1]
[1 0 1 0 0 1 1]
[0 0 0 0 0 1 1]
[0 0 0 0 0 1 1]
[1 0 1 0 0 1 1]
[1 0 0 0 0 1 1]
[1 0 0 0 0 1 1]
[0 1 0 0 0 1 1]
[1 0 1 0 0 1 1]
[0 0 0 0 0 1 1]
[0 0 0 0 0 1 1]
[0 0 1 0 0 1 1]
[0 0 0 0 0 1 1]
[0 0 0 0 0 1 1]
[1 0 0 0 0 1 1]
[1 0 1 0 0 1 1]
[0 0 0 0 0 1 1]
[0 0 0 0 0 1 1]
[0 0 1 0 0 0 1]
[0 0 0 0 0 1 1]
[0 1 0 0 0 1 1]
[1 0 0 0 0 1 1]
[1 0 0 0 0 1 1]
[0 0 0 0 0 1 1]
[0 0 0 0 0 1 1]
[0 0 1 0 0 0 1]
[0 0 0 0 0 1 1]
[0 0 1 0 0 1 0]
[0 1 0 0 0 1 1]
[1 0 0 0 0 1 1]
[1 0 1 0 0 1 1]
[0 0 0 0 0 1 1]
[0 0 0 0 0 1 1]
[0 0 1 0 0 1 0]
[0 0 1 0 0 1 0]
[0 0 1 0 0 1 0]
[0 0 1 0 0 1 0]
[0 0 1 0 0 1 0]
[0 0 1 0 0 1 0]
[0 0 1 0 0 1 0]


In [226]:
for i in range(1,10):
    print(i)

1
2
3
4
5
6
7
8
9
